# California Drinking Water Data and Water Filters
an exploratory data analysis

## Dataset background

This dataset was extracted from the state of California's Open Data website: https://data.ca.gov/dataset/water-quality-data

Supplemental data regarding California State Maximums is from:


### lab results

https://data.ca.gov/dataset/water-quality-data/resource/084f4e83-3d44-42ed-badb-ab52ee74ce5a

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
calabs = pd.read_csv('lab-results.csv')
calabs.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361130 entries, 0 to 4361129
Data columns (total 18 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   STATION_ID          4361130 non-null  int64  
 1   STATION_NAME        4361130 non-null  object 
 2   FULL_STATION_NAME   4361130 non-null  object 
 3   STATION_NUMBER      4361130 non-null  object 
 4   STATION_TYPE        4361130 non-null  object 
 5   LATITUDE            4236754 non-null  float64
 6   LONGITUDE           4236754 non-null  float64
 7   STATUS              4361130 non-null  object 
 8   COUNTY_NAME         4361130 non-null  object 
 9   SAMPLE_CODE         4361130 non-null  object 
 10  SAMPLE_DATE         4361130 non-null  object 
 11  SAMPLE_DEPTH        1587155 non-null  float64
 12  SAMPLE_DEPTH_UNITS  4361130 non-null  object 
 13  PARAMETER           4361130 non-null  object 
 14  RESULT              4325544 non-null  float64
 15  REPORTING_LIMIT

In [3]:
# What's the timeframe?
calabs['SAMPLE_DATE'] = pd.to_datetime(calabs['SAMPLE_DATE'])
print('test result dates range from', calabs.SAMPLE_DATE.min(),'to', calabs.SAMPLE_DATE.max())

test result dates range from 1903-03-23 00:00:00 to 2020-07-01 12:12:00


In [4]:
# any NANs?
calabs.isnull().sum()

STATION_ID                  0
STATION_NAME                0
FULL_STATION_NAME           0
STATION_NUMBER              0
STATION_TYPE                0
LATITUDE               124376
LONGITUDE              124376
STATUS                      0
COUNTY_NAME                 0
SAMPLE_CODE                 0
SAMPLE_DATE                 0
SAMPLE_DEPTH          2773975
SAMPLE_DEPTH_UNITS          0
PARAMETER                   0
RESULT                  35586
REPORTING_LIMIT         17300
UNITS                       0
METHOD_NAME                 0
dtype: int64

In [5]:
# what stations have NAN LAT LONG values? 
null_lat = calabs[calabs.LATITUDE.isnull()]
print(f'number of stations with NANs:', null_lat.STATION_ID.nunique())
print(f'type of stations with NANs:', null_lat.STATION_TYPE.value_counts())

number of stations with NANs: 85
type of stations with NANs: Other    124376
Name: STATION_TYPE, dtype: int64


since the station type is "Other" and not specific to the type of water sampled, these rows offer little value and can be dropped.

In [6]:
# Review information for results with NANs. Can these rows be dropped?
result_nan = calabs[(calabs.RESULT.isnull())]
result_nan.PARAMETER.value_counts()[:5].sort_values(ascending=False)

*No Lab Analyses (Field Measures Only)    17215
Dissolved Sulfate                           501
Dissolved Chloride                          500
Total Dissolved Solids                      498
Dissolved Boron                             495
Name: PARAMETER, dtype: int64

The majority parameter "No Lab Analyses" indicates that these rows can be dropped. And, there is another dataset for Field Measures that should be merged.

### Field Measures
https://data.ca.gov/dataset/water-quality-data/resource/5eeaf27e-315d-4b95-9c34-cf63d168e8f5

In [7]:
cafield = pd.read_csv('field-results.csv')
cafield.info(null_counts = True)

/Users/kathanner/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1105915 entries, 0 to 1105914
Data columns (total 23 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   STATION_ID           1105915 non-null  int64  
 1   STATION_NAME         1105915 non-null  object 
 2   STATION_NUMBER       1105915 non-null  object 
 3   FULL_STATION_NAME    1105915 non-null  object 
 4   STATION_TYPE         1105915 non-null  object 
 5   LATITUDE             1105915 non-null  float64
 6   LONGITUDE            1105915 non-null  float64
 7   STATUS               1105915 non-null  object 
 8   COUNTY_NAME          1105915 non-null  object 
 9   SAMPLE_CODE          1105915 non-null  object 
 10  FDR_CODE             1105915 non-null  object 
 11  SAMPLE_DATE          1105915 non-null  object 
 12  SAMPLE_DEPTH         543157 non-null   float64
 13  SAMPLE_DEPTH_UNITS   1105915 non-null  object 
 14  ANL_DATA_TYPE        129953 non-null   object 
 15

In [8]:
# What's the timeframe?
cafield['SAMPLE_DATE'] = pd.to_datetime(cafield['SAMPLE_DATE'])
print('test result dates range from', cafield.SAMPLE_DATE.min(),'to', cafield.SAMPLE_DATE.max())

test result dates range from 1913-05-13 13:30:00 to 2020-07-10 09:00:00


In [9]:
# Any NANs?
cafield.isnull().sum()

STATION_ID                   0
STATION_NAME                 0
STATION_NUMBER               0
FULL_STATION_NAME            0
STATION_TYPE                 0
LATITUDE                     0
LONGITUDE                    0
STATUS                       0
COUNTY_NAME                  0
SAMPLE_CODE                  0
FDR_CODE                     0
SAMPLE_DATE                  0
SAMPLE_DEPTH            562758
SAMPLE_DEPTH_UNITS           0
ANL_DATA_TYPE           975962
PARAMETER                    0
FDR_RESULT              124002
FDR_TEXT_RESULT         985614
FDR_DATE_RESULT        1102729
FDR_REPORTING_LIMIT     151510
UNS_NAME                     0
MTH_NAME                     0
FDR_FOOTNOTE            991012
dtype: int64

Assumption: the missing field results have a corresponding lab result. Drop NAN rows.

### notes:

For this model, the following are unnecessary:

Lab Results:
* results recorded before 2010
* LATITUDE and LONGITUDE NAN rows
* RESULT NAN rows
* FULL_STATION_NAME
* STATION_NUMBER
* STATUS
* SAMPLE_CODE
* SAMPLE_DEPTH
* SAMPLE_DEPTH_UNITS
* SAMPLE_DATE
* REPORTING_LIMIT

Field Measures:
* results recorded before 2010
* RESULT NAN rows
* STATION_NUMBER
* FULL_STATION_NAME
* STATUS
* SAMPLE_CODE
* FDR_CODE 
* SAMPLE_DATE  
* SAMPLE_DEPTH       
* SAMPLE_DEPTH_UNITS
* ANL_DATA_TYPE
* FDR_TEXT_RESULT 
* FDR_DATE_RESULT  
* FDR_REPORTING_LIMIT 
* FDR_FOOTNOTE 

## Data Cleaning

In [10]:
# remove samples before 2010
calabs['SAMPLE_YEAR'] = pd.DatetimeIndex(calabs['SAMPLE_DATE']).year
calabs = calabs[calabs.SAMPLE_YEAR >= 2010]

cafield['SAMPLE_YEAR'] = pd.DatetimeIndex(cafield['SAMPLE_DATE']).year
cafield = cafield[cafield.SAMPLE_YEAR >= 2010]

In [11]:
# remove NAN rows in LATITUDE, LONGITUDE and RESULT
calabs = calabs.dropna(subset=['LATITUDE', 'LONGITUDE', 'RESULT'])
cafield = cafield.dropna(subset=['FDR_RESULT'])

In [12]:
# drop unnecessary columns
calabs = calabs.drop(['FULL_STATION_NAME', 'STATION_NUMBER', 'STATUS',
                      'SAMPLE_CODE', 'SAMPLE_DEPTH','SAMPLE_DEPTH_UNITS',
                      'REPORTING_LIMIT','SAMPLE_DATE'], axis=1)
cafield = cafield.drop(['STATION_NUMBER', 'FULL_STATION_NAME', 'STATUS',
                        'SAMPLE_CODE', 'FDR_CODE', 'SAMPLE_DATE',
                        'FDR_TEXT_RESULT', 'FDR_DATE_RESULT',
                        'SAMPLE_DEPTH', 'SAMPLE_DEPTH_UNITS', 'ANL_DATA_TYPE', 
                        'FDR_REPORTING_LIMIT', 'FDR_FOOTNOTE'], axis=1)

In [13]:
calabs.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366296 entries, 12694 to 4357924
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   STATION_ID    366296 non-null  int64  
 1   STATION_NAME  366296 non-null  object 
 2   STATION_TYPE  366296 non-null  object 
 3   LATITUDE      366296 non-null  float64
 4   LONGITUDE     366296 non-null  float64
 5   COUNTY_NAME   366296 non-null  object 
 6   PARAMETER     366296 non-null  object 
 7   RESULT        366296 non-null  float64
 8   UNITS         366296 non-null  object 
 9   METHOD_NAME   366296 non-null  object 
 10  SAMPLE_YEAR   366296 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 33.5+ MB


In [14]:
cafield.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118449 entries, 4733 to 1105697
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   STATION_ID    118449 non-null  int64  
 1   STATION_NAME  118449 non-null  object 
 2   STATION_TYPE  118449 non-null  object 
 3   LATITUDE      118449 non-null  float64
 4   LONGITUDE     118449 non-null  float64
 5   COUNTY_NAME   118449 non-null  object 
 6   PARAMETER     118449 non-null  object 
 7   FDR_RESULT    118449 non-null  float64
 8   UNS_NAME      118449 non-null  object 
 9   MTH_NAME      118449 non-null  object 
 10  SAMPLE_YEAR   118449 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 10.8+ MB


In [15]:
# concatenate datasets
cafield = cafield.rename(columns={"FDR_RESULT":"RESULT","UNS_NAME":"UNITS",
                                  "MTH_NAME":"METHOD_NAME"})
caopendata = pd.concat([calabs, cafield], ignore_index=True)
caopendata.head()

,STATION_ID,STATION_NAME,STATION_TYPE,LATITUDE,LONGITUDE,COUNTY_NAME,PARAMETER,RESULT,UNITS,METHOD_NAME,SAMPLE_YEAR
0,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1,1,2-Tetrachloroethane",0.0,ug/L,EPA 524.2,2016
1,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1,1-Trichloroethane",0.0,ug/L,EPA 524.2,2016
2,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1,2,2-Tetrachloroethane",0.0,ug/L,EPA 524.2,2016
3,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1,2-Trichloroethane",0.0,ug/L,EPA 524.2,2016
4,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1-Dichloroethane",0.0,ug/L,EPA 524.2,2016


In [16]:
caopendata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484745 entries, 0 to 484744
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   STATION_ID    484745 non-null  int64  
 1   STATION_NAME  484745 non-null  object 
 2   STATION_TYPE  484745 non-null  object 
 3   LATITUDE      484745 non-null  float64
 4   LONGITUDE     484745 non-null  float64
 5   COUNTY_NAME   484745 non-null  object 
 6   PARAMETER     484745 non-null  object 
 7   RESULT        484745 non-null  float64
 8   UNITS         484745 non-null  object 
 9   METHOD_NAME   484745 non-null  object 
 10  SAMPLE_YEAR   484745 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 40.7+ MB


## Data Merge with California Limits and CDC Filter Types

In [17]:
# export the parameters included in the Open Data
parameters = caopendata.PARAMETER.value_counts()
parameters.to_csv('param.csv')

### California State Water Resources Control Board
referencing the California maximum contaminant levels (MCLs) published here:
https://www.waterboards.ca.gov/drinking_water/certlic/drinkingwater/MCLsandPHGs.html

Open Data Parameters recorded and contaminants monitored by California:
* 	1,1-Dichloroethane
* 	1,1,1-Trichloroethane
* 	1,1,2-Trichloroethane
* 	1,1,2,2-Tetrachloroethane
* 	1,2-Dibromo-3-chloropropane (DBCP)
* 	1,2-Dichlorobenzene
* 	1,2-Dichloroethane
* 	1,2-Dichloropropane
* 	1,2,3-Trichloropropane
* 	1,2,4-Trichlorobenzene
* 	1,4-Dichlorobenzene
* 	2,4,5-TP (Silvex)
* 	Alachlor
* 	Aluminum
* 	Antimony
* 	Arsenic
* 	Atrazine
* 	Barium
* 	Benzene
* 	Beryllium
* 	BHC-gamma (Lindane)
* 	Cadmium
* 	Carbofuran
* 	Carbon tetrachloride
* 	Chlordane
* 	Chromium
* 	cis-1,3-Dichloropropene
* 	Copper
* 	Dinoseb (DNPB)
* 	Endrin
* 	Fluoride
* 	Glyphosate
* 	Heptachlor
* 	Heptachlor epoxide
* 	Lead
* 	Mercury
* 	Methoxychlor
* 	Molinate
* 	Nickel
* 	Nitrate
* 	Nitrate + Nitrite
* 	Nitrite
* 	Oxamyl
* 	Pentachlorophenol (PCP)
* 	Picloram
* 	Selenium
* 	Simazine
* 	Strontium
* 	Styrene
* 	Thallium
* 	Thiobencarb
* 	Toluene
* 	Toxaphene
* 	trans-1,3-Dichloropropene
* 	Trichlorofluoromethane
* 	Vinyl chloride

The MCL for Hardness and Turbidity is a personal preference. To calculate the MCLs for these parameter categories, the average result by station ID has been added.

In [18]:
# merge CA maximum contaminant levels (MCLs)
cacontrol = pd.read_csv('mcl.csv')
caopendata['PARAMETER'] = caopendata['PARAMETER'].str.strip()
cawater = pd.merge(caopendata, cacontrol, on='PARAMETER', how='outer')

In [19]:
cawater[cawater.PARAMETER == 'Heptachlor'].head()

,STATION_ID,STATION_NAME,STATION_TYPE,LATITUDE,LONGITUDE,COUNTY_NAME,PARAMETER,RESULT,UNITS,METHOD_NAME,SAMPLE_YEAR,MCL
377821,12,H.O. Banks Headworks,Surface Water,37.8019,-121.62,Alameda,Heptachlor,0.0,ug/L,EPA 608,2016,0.00001
377822,12,H.O. Banks Headworks,Surface Water,37.8019,-121.62,Alameda,Heptachlor,0.0,ug/L,EPA 608,2011,0.00001
377823,12,H.O. Banks Headworks,Surface Water,37.8019,-121.62,Alameda,Heptachlor,0.0,ug/L,EPA 608,2010,0.00001
377824,12,H.O. Banks Headworks,Surface Water,37.8019,-121.62,Alameda,Heptachlor,0.0,ug/L,EPA 608,2015,0.00001
377825,12,H.O. Banks Headworks,Surface Water,37.8019,-121.62,Alameda,Heptachlor,0.0,ug/L,EPA 608,2014,0.00001


In [20]:
cawater.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 484745 entries, 0 to 484744
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   STATION_ID    484745 non-null  int64  
 1   STATION_NAME  484745 non-null  object 
 2   STATION_TYPE  484745 non-null  object 
 3   LATITUDE      484745 non-null  float64
 4   LONGITUDE     484745 non-null  float64
 5   COUNTY_NAME   484745 non-null  object 
 6   PARAMETER     484745 non-null  object 
 7   RESULT        484745 non-null  float64
 8   UNITS         484745 non-null  object 
 9   METHOD_NAME   484745 non-null  object 
 10  SAMPLE_YEAR   484745 non-null  int64  
 11  MCL           106586 non-null  float64
dtypes: float64(4), int64(2), object(6)
memory usage: 48.1+ MB


In [21]:
import sqlite3
sqlite3.connect('water.db')
conn = sqlite3.connect('water.db')  
cur = conn.cursor()

cur.execute('''CREATE TABLE cawater
                ([STATION_ID] INTEGER
                );
            ''')
conn.commit()
cawater.to_sql('cawater', conn, if_exists='replace', index = False)

In [22]:
cur.execute(""" 
SELECT c.STATION_ID, AVG(c.RESULT) as TURBIDITY_AVG
FROM cawater c
WHERE (c.PARAMETER == 'Turbidity')
GROUP BY c.STATION_ID
;""")

tur = pd.DataFrame(cur.fetchall())
tur.columns = (x[0] for x in cur.description)
print(tur.shape)
print(tur.info())
tur.head()

(498, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STATION_ID     498 non-null    int64  
 1   TURBIDITY_AVG  498 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 7.9 KB
None


,STATION_ID,TURBIDITY_AVG
0,1,3.093066
1,10,8.259178
2,12,8.957236
3,73,41.547391
4,80,5.620538


In [23]:
cawater = pd.merge(cawater, tur, on='STATION_ID', how='outer')

In [24]:
cur.execute(""" 
SELECT c.STATION_ID, AVG(c.RESULT) as HARDNESS_AVG
FROM cawater c
WHERE (c.PARAMETER like '%Hardness%')
GROUP BY c.STATION_ID
;""")

hard = pd.DataFrame(cur.fetchall())
hard.columns = (x[0] for x in cur.description)
print(hard.shape)
print(hard.info())
hard.head()

(593, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STATION_ID    593 non-null    int64  
 1   HARDNESS_AVG  593 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 9.4 KB
None


,STATION_ID,HARDNESS_AVG
0,1,23.747899
1,10,89.227642
2,12,90.042553
3,73,188.545455
4,80,101.576000


In [25]:
cawater = pd.merge(cawater, hard, on='STATION_ID', how='outer')

In [26]:
# for TURBIDITY and HARDNESS, assign average result to MCL by STATION_ID
cawater.loc[cawater['PARAMETER'].str.contains('Turbidity'), 'MCL'] = cawater.TURBIDITY_AVG
cawater.loc[cawater['PARAMETER'].str.contains('Hardness'), 'MCL'] = cawater.HARDNESS_AVG

In [27]:
# drop redundant columns and records that California is not monitoring
cawater = cawater.drop(['TURBIDITY_AVG','HARDNESS_AVG'], axis = 1)
cawater = cawater.dropna(subset=['MCL'])

In [28]:
cawater.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137212 entries, 24 to 477143
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   STATION_ID    137212 non-null  int64  
 1   STATION_NAME  137212 non-null  object 
 2   STATION_TYPE  137212 non-null  object 
 3   LATITUDE      137212 non-null  float64
 4   LONGITUDE     137212 non-null  float64
 5   COUNTY_NAME   137212 non-null  object 
 6   PARAMETER     137212 non-null  object 
 7   RESULT        137212 non-null  float64
 8   UNITS         137212 non-null  object 
 9   METHOD_NAME   137212 non-null  object 
 10  SAMPLE_YEAR   137212 non-null  int64  
 11  MCL           137212 non-null  float64
dtypes: float64(4), int64(2), object(6)
memory usage: 13.6+ MB


### CDC Water Filter Information
https://www.cdc.gov/healthywater/drinking/home-water-treatment/water-filters/step3.html

In [29]:
# add type of filters
conditions = [((cawater['RESULT'] > (cawater['MCL'] + (cawater['MCL']*.5)))),
              (cawater['PARAMETER'].str.contains('Copper') & (cawater['RESULT'] > cawater['MCL'])
               | cawater['PARAMETER'].str.contains('Lead') & (cawater['RESULT'] > cawater['MCL'])), 
              (cawater['PARAMETER'].str.contains('Hardness') & (cawater['RESULT'] > cawater['MCL'])
               | cawater['PARAMETER'].str.contains('Barium') & (cawater['RESULT'] > cawater['MCL'])
               | cawater['PARAMETER'].str.contains('Fluoride') & (cawater['RESULT'] > cawater['MCL'])),
              (cawater['PARAMETER'].str.contains('Turbidity') & (cawater['RESULT'] > cawater['MCL'])
               | cawater['PARAMETER'].str.contains('Nitrate') & (cawater['RESULT'] > cawater['MCL'])
               | cawater['PARAMETER'].str.contains('Nitrite') & (cawater['RESULT'] > cawater['MCL']))]
choices = ["Distillation","Activated Carbon Filter", "Ion Exchange", "Reverse Osmosis"]
#len(conditions)
cawater['filters'] = np.select(conditions, choices, default="Activated Carbon Filter")

In [30]:
# update water.db
cawater.to_sql('cawater', conn, if_exists='replace', index = False)

## Year over Year contaminants 

In [31]:
# cawater county plot

## Features for base model

### base model with regions
California census data shows 10 regions: https://census.ca.gov/regions/

In [32]:
sc = ['Butte', 'Colusa', 'El Dorado', 'Glenn', 'Lassen', 'Modoc', 'Nevada', 
      'Placer', 'Plumas', 'Sacramento', 'Shasta', 'Sierra', 'Siskiyou', 
      'Sutter', 'Tehama', 'Yolo', 'Yuba']
nc = ['Del Norte', 'Humboldt', 'Lake', 'Mendocino', 'Napa', 'Sonoma', 'Trinity']
sf = ['Alameda', 'Contra Costa', 'Marin', 'San Francisco', 'San Mateo', 'Santa Clara', 'Solano']
nsj = ['Alpine', 'Amador', 'Calaveras', 'Madera', 'Mariposa', 'Merced', 'Mono', 'San Joaquin', 'Stanislaus', 'Tuolumne']
cc = ['Monterey', 'San Benito', 'San Luis Obispo', 'Santa Barbara', 'Santa Cruz', 'Ventura']
ssj = ['Fresno', 'Inyo', 'Kern', 'Kings', 'Tulare']
ie = ['Riverside', 'San Bernardino']
la = ['Los Angeles']
oc = ['Orange']
sd = ['Imperial', 'San Diego']
cawater['Superior_California'] = np.where(cawater['COUNTY_NAME'].isin(sc), 1, 0)
cawater['North_Coast'] = np.where(cawater['COUNTY_NAME'].isin(nc), 1, 0)
cawater['San_Francisco'] = np.where(cawater['COUNTY_NAME'].isin(sf), 1, 0)
cawater['Northern_San_Joaquin_Valley'] = np.where(cawater['COUNTY_NAME'].isin(nsj), 1, 0)
cawater['Central_Coast'] = np.where(cawater['COUNTY_NAME'].isin(cc), 1, 0)
cawater['Southern_San_Joaquin_Valley'] = np.where(cawater['COUNTY_NAME'].isin(ssj), 1, 0)
cawater['Inland_Empire'] = np.where(cawater['COUNTY_NAME'].isin(ie), 1, 0)
cawater['Los_Angeles'] = np.where(cawater['COUNTY_NAME'].isin(la), 1, 0)
cawater['Orange_County'] = np.where(cawater['COUNTY_NAME'].isin(oc), 1, 0)
cawater['San_Diego'] = np.where(cawater['COUNTY_NAME'].isin(sd), 1, 0)

### remapping parameter and result

In [33]:
cawater['1,1-Dichloroethane'] = np.where(cawater['PARAMETER'].str.contains('1,1-Dichloroethane'), cawater['RESULT'], 0)
cawater['1,1,1-Trichloroethane'] = np.where(cawater['PARAMETER'].str.contains('1,1,1-Trichloroethane'), cawater['RESULT'], 0)
cawater['1,1,2-Trichloroethane'] = np.where(cawater['PARAMETER'].str.contains('1,1,2-Trichloroethane'), cawater['RESULT'], 0)
cawater['1,1,2,2-Tetrachloroethane'] = np.where(cawater['PARAMETER'].str.contains('1,1,2,2-Tetrachloroethane'), cawater['RESULT'], 0)
cawater['1,2-Dibromo-3-chloropropane (DBCP)'] = np.where(cawater['PARAMETER'].str.contains('1,2-Dibromo-3-chloropropane (DBCP)'), cawater['RESULT'], 0)
cawater['1,2-Dichlorobenzene'] = np.where(cawater['PARAMETER'].str.contains('1,2-Dichlorobenzene'), cawater['RESULT'], 0)
cawater['1,2-Dichloroethane'] = np.where(cawater['PARAMETER'].str.contains('1,2-Dichloroethane'), cawater['RESULT'], 0)
cawater['1,2-Dichloropropane'] = np.where(cawater['PARAMETER'].str.contains('1,2-Dichloropropane'), cawater['RESULT'], 0)
cawater['1,2,3-Trichloropropane'] = np.where(cawater['PARAMETER'].str.contains('1,2,3-Trichloropropane'), cawater['RESULT'], 0)
cawater['1,2,4-Trichlorobenzene'] = np.where(cawater['PARAMETER'].str.contains('1,2,4-Trichlorobenzene'), cawater['RESULT'], 0)
cawater['1,4-Dichlorobenzene'] = np.where(cawater['PARAMETER'].str.contains('1,4-Dichlorobenzene'), cawater['RESULT'], 0)
cawater['2,4,5-TP (Silvex)'] = np.where(cawater['PARAMETER'].str.contains('2,4,5-TP (Silvex)'), cawater['RESULT'], 0)
cawater['Alachlor'] = np.where(cawater['PARAMETER'].str.contains('Alachlor'), cawater['RESULT'], 0)
cawater['Aluminum'] = np.where(cawater['PARAMETER'].str.contains('Aluminum'), cawater['RESULT'], 0)
cawater['Antimony'] = np.where(cawater['PARAMETER'].str.contains('Antimony'), cawater['RESULT'], 0)
cawater['Arsenic'] = np.where(cawater['PARAMETER'].str.contains('Arsenic'), cawater['RESULT'], 0)
cawater['Atrazine'] = np.where(cawater['PARAMETER'].str.contains('Atrazine'), cawater['RESULT'], 0)
cawater['Barium'] = np.where(cawater['PARAMETER'].str.contains('Barium'), cawater['RESULT'], 0)
cawater['Benzene'] = np.where(cawater['PARAMETER'].str.contains('Benzene'), cawater['RESULT'], 0)
cawater['Beryllium'] = np.where(cawater['PARAMETER'].str.contains('Beryllium'), cawater['RESULT'], 0)
cawater['BHC-gamma (Lindane)'] = np.where(cawater['PARAMETER'].str.contains('BHC-gamma (Lindane)'), cawater['RESULT'], 0)
cawater['Cadmium'] = np.where(cawater['PARAMETER'].str.contains('Cadmium'), cawater['RESULT'], 0)
cawater['Carbofuran'] = np.where(cawater['PARAMETER'].str.contains('Carbofuran'), cawater['RESULT'], 0)
cawater['Carbon tetrachloride'] = np.where(cawater['PARAMETER'].str.contains('Carbon tetrachloride'), cawater['RESULT'], 0)
cawater['Chlordane'] = np.where(cawater['PARAMETER'].str.contains('Chlordane'), cawater['RESULT'], 0)
cawater['Chromium'] = np.where(cawater['PARAMETER'].str.contains('Chromium'), cawater['RESULT'], 0)
cawater['cis-1,3-Dichloropropene'] = np.where(cawater['PARAMETER'].str.contains('cis-1,3-Dichloropropene'), cawater['RESULT'], 0)
cawater['Copper'] = np.where(cawater['PARAMETER'].str.contains('Copper'), cawater['RESULT'], 0)
cawater['Dinoseb (DNPB)'] = np.where(cawater['PARAMETER'].str.contains('Dinoseb (DNPB)'), cawater['RESULT'], 0)
cawater['Endrin'] = np.where(cawater['PARAMETER'].str.contains('Endrin'), cawater['RESULT'], 0)
cawater['Fluoride'] = np.where(cawater['PARAMETER'].str.contains('Fluoride'), cawater['RESULT'], 0)
cawater['Glyphosate'] = np.where(cawater['PARAMETER'].str.contains('Glyphosate'), cawater['RESULT'], 0)
cawater['Heptachlor'] = np.where(cawater['PARAMETER'].str.contains('Heptachlor'), cawater['RESULT'], 0)
cawater['Heptachlor epoxide'] = np.where(cawater['PARAMETER'].str.contains('Heptachlor epoxide'), cawater['RESULT'], 0)
cawater['Lead'] = np.where(cawater['PARAMETER'].str.contains('Lead'), cawater['RESULT'], 0)
cawater['Mercury'] = np.where(cawater['PARAMETER'].str.contains('Mercury'), cawater['RESULT'], 0)
cawater['Methoxychlor'] = np.where(cawater['PARAMETER'].str.contains('Methoxychlor'), cawater['RESULT'], 0)
cawater['Molinate'] = np.where(cawater['PARAMETER'].str.contains('Molinate'), cawater['RESULT'], 0)
cawater['Nickel'] = np.where(cawater['PARAMETER'].str.contains('Nickel'), cawater['RESULT'], 0)
cawater['Nitrate'] = np.where(cawater['PARAMETER'].str.contains('Nitrate'), cawater['RESULT'], 0)
cawater['Nitrate + Nitrite'] = np.where(cawater['PARAMETER'].str.contains('Nitrate + Nitrite'), cawater['RESULT'], 0)
cawater['Nitrite'] = np.where(cawater['PARAMETER'].str.contains('Nitrite'), cawater['RESULT'], 0)
cawater['Oxamyl'] = np.where(cawater['PARAMETER'].str.contains('Oxamyl'), cawater['RESULT'], 0)
cawater['Pentachlorophenol (PCP)'] = np.where(cawater['PARAMETER'].str.contains('Pentachlorophenol (PCP)'), cawater['RESULT'], 0)
cawater['Picloram'] = np.where(cawater['PARAMETER'].str.contains('Picloram'), cawater['RESULT'], 0)
cawater['Selenium'] = np.where(cawater['PARAMETER'].str.contains('Selenium'), cawater['RESULT'], 0)
cawater['Simazine'] = np.where(cawater['PARAMETER'].str.contains('Simazine'), cawater['RESULT'], 0)
cawater['Strontium'] = np.where(cawater['PARAMETER'].str.contains('Strontium'), cawater['RESULT'], 0)
cawater['Styrene'] = np.where(cawater['PARAMETER'].str.contains('Styrene'), cawater['RESULT'], 0)
cawater['Thallium'] = np.where(cawater['PARAMETER'].str.contains('Thallium'), cawater['RESULT'], 0)
cawater['Thiobencarb'] = np.where(cawater['PARAMETER'].str.contains('Thiobencarb'), cawater['RESULT'], 0)
cawater['Toluene'] = np.where(cawater['PARAMETER'].str.contains('Toluene'), cawater['RESULT'], 0)
cawater['Toxaphene'] = np.where(cawater['PARAMETER'].str.contains('Toxaphene'), cawater['RESULT'], 0)
cawater['trans-1,3-Dichloropropene'] = np.where(cawater['PARAMETER'].str.contains('trans-1,3-Dichloropropene'), cawater['RESULT'], 0)
cawater['Trichlorofluoromethane'] = np.where(cawater['PARAMETER'].str.contains('Trichlorofluoromethane'), cawater['RESULT'], 0)
cawater['Vinyl chloride'] = np.where(cawater['PARAMETER'].str.contains('Vinyl chloride'), cawater['RESULT'], 0)

/Users/kathanner/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


### Type of water

In [34]:
cawater = pd.concat([cawater, pd.get_dummies(cawater['STATION_TYPE'])], axis=1)
cawater.head()

,STATION_ID,STATION_NAME,STATION_TYPE,LATITUDE,LONGITUDE,COUNTY_NAME,PARAMETER,RESULT,UNITS,METHOD_NAME,SAMPLE_YEAR,MCL,filters,Superior_California,North_Coast,San_Francisco,Northern_San_Joaquin_Valley,Central_Coast,Southern_San_Joaquin_Valley,Inland_Empire,Los_Angeles,Orange_County,San_Diego,"1,1-Dichloroethane","1,1,1-Trichloroethane","1,1,2-Trichloroethane","1,1,2,2-Tetrachloroethane","1,2-Dibromo-3-chloropropane (DBCP)","1,2-Dichlorobenzene","1,2-Dichloroethane","1,2-Dichloropropane","1,2,3-Trichloropropane","1,2,4-Trichlorobenzene","1,4-Dichlorobenzene","2,4,5-TP (Silvex)",Alachlor,Aluminum,Antimony,Arsenic,Atrazine,Barium,Benzene,Beryllium,BHC-gamma (Lindane),Cadmium,Carbofuran,Carbon tetrachloride,Chlordane,Chromium,"cis-1,3-Dichloropropene",Copper,Dinoseb (DNPB),Endrin,Fluoride,Glyphosate,Heptachlor,Heptachlor epoxide,Lead,Mercury,Methoxychlor,Molinate,Nickel,Nitrate,Nitrate + Nitrite,Nitrite,Oxamyl,Pentachlorophenol (PCP),Picloram,Selenium,Simazine,Strontium,Styrene,Thallium,Thiobencarb,Toluene,Toxaphene,"trans-1,3-Dichloropropene",Trichlorofluoromethane,Vinyl chloride,Groundwater,Other,Surface Water
24,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1,1-Trichloroethane",0.0,ug/L,EPA 524.2,2016,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
25,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1,1-Trichloroethane",0.0,ug/L,EPA 502.2,2012,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
26,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1,1-Trichloroethane",0.0,ug/L,EPA 524.2,2016,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
27,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1,1-Trichloroethane",0.0,ug/L,EPA 524.2,2019,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
28,259,Del Valle COW,Surface Water,37.6183,-121.749,Alameda,"1,1,1-Trichloroethane",0.0,ug/L,EPA 502.2,2012,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1


In [35]:
# drop unnecessary columns
base = cawater.drop(['STATION_TYPE','STATION_ID','STATION_NAME','LATITUDE',
                     'LONGITUDE','COUNTY_NAME','PARAMETER','RESULT','UNITS','METHOD_NAME',
                     'Orange_County','San_Diego'], axis = 1) # note: no data from OC or SD

In [36]:
# export for modeling
base.to_csv('base.csv', index=False)

In [37]:
# quick check
base.head()

,SAMPLE_YEAR,MCL,filters,Superior_California,North_Coast,San_Francisco,Northern_San_Joaquin_Valley,Central_Coast,Southern_San_Joaquin_Valley,Inland_Empire,Los_Angeles,"1,1-Dichloroethane","1,1,1-Trichloroethane","1,1,2-Trichloroethane","1,1,2,2-Tetrachloroethane","1,2-Dibromo-3-chloropropane (DBCP)","1,2-Dichlorobenzene","1,2-Dichloroethane","1,2-Dichloropropane","1,2,3-Trichloropropane","1,2,4-Trichlorobenzene","1,4-Dichlorobenzene","2,4,5-TP (Silvex)",Alachlor,Aluminum,Antimony,Arsenic,Atrazine,Barium,Benzene,Beryllium,BHC-gamma (Lindane),Cadmium,Carbofuran,Carbon tetrachloride,Chlordane,Chromium,"cis-1,3-Dichloropropene",Copper,Dinoseb (DNPB),Endrin,Fluoride,Glyphosate,Heptachlor,Heptachlor epoxide,Lead,Mercury,Methoxychlor,Molinate,Nickel,Nitrate,Nitrate + Nitrite,Nitrite,Oxamyl,Pentachlorophenol (PCP),Picloram,Selenium,Simazine,Strontium,Styrene,Thallium,Thiobencarb,Toluene,Toxaphene,"trans-1,3-Dichloropropene",Trichlorofluoromethane,Vinyl chloride,Groundwater,Other,Surface Water
24,2016,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
25,2012,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
26,2016,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
27,2019,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
28,2012,0.2,Activated Carbon Filter,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
